In [ ]:
pip install scikit-optimize

In [3]:
import warnings
warnings.filterwarnings("ignore")
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

db = pd.read_csv('Dizyme.csv')

di2 = {'TMB':1.0, 'H2O2':2.0, 'ABTS': 3.0, 'OPD': 4.0, 'DAB':5.0, 'BA':6.0}
db['Subtype'].replace(di2, inplace=True)


In [ ]:
x = db.loc[:,'Syngony':'volume'].values
y = db.loc[:,'Kcat'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7)

from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler(feature_range=(0, 1))
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

y_train = np.log10(y_train)
y_test = np.log10(y_test)

In [ ]:
reg = RandomForestRegressor(criterion='mae')

search_space = {
    "n_estimators": Integer(100,5000),
    "min_samples_split": Integer(2, 30),
    "min_samples_leaf": Integer(1, 30),
    "max_depth": Integer(1, 100),
    "max_features": Categorical(['auto', 'sqrt']),
}

reg_bayes = BayesSearchCV(
    estimator=reg,
    search_spaces=search_space,
    cv=5,
    verbose=2,
    n_jobs=-1
)
reg_bayes.fit(x_train,y_train)
n_esti = reg_bayes.best_estimator_.n_estimators
depth = reg_bayes.best_estimator_.max_depth
min_leaf = reg_bayes.best_estimator_.min_samples_leaf
min_split = reg_bayes.best_estimator_.min_samples_split
max_feat = reg_bayes.best_estimator_.max_features

print('n_esti:', n_esti,'depth:', depth,'min_leaf:', min_leaf,'min_split:', min_split,'max_feat:', max_feat)
